In [1]:
data_var = ''

In [2]:
# Parameters
data_var = "2024-07-14"


In [3]:
import pandas as pd; pd.set_option('display.max_columns', None)
import os
from pycaret.classification import *
from scripts.PreparaDatasetV2 import *
from scripts.TeoriaRetornos import *

### Funções

In [4]:
def atualiza_dia(_jogos):
    _jogos = prepara_df(_jogos)

    _jogos = _jogos.reset_index(drop=True)

    return _jogos

### Pegando os jogos do dia

In [5]:
df_today = pd.read_csv(f'jogos_do_dia/{data_var}.csv')

### Atualizando os datasets que tem jogo hoje

In [6]:
temporada = pd.read_csv('data/Entire_Season.csv')
temporada = temporada[temporada['Date'] < data_var]
temporada = pd.concat([temporada, df_today])
ligas = df_today['League'].unique().tolist()
temporada = temporada[temporada['League'].isin(ligas)]
temporada.reset_index(drop=True, inplace=True)

### Cria variáveis na temporada e nos jogos do dia

In [7]:
games = atualiza_dia(temporada)
temporada = games[games['Date'] < data_var]
os.makedirs(f'apostas_do_dia/{data_var}', exist_ok=True)

In [8]:
jogos = games[games['Date'] == data_var]

jogos = jogos.drop(columns=['Home_Pts', 'Away_Pts',
       'Back_Home', 'PL_Home', 'Back_Away',
       'PL_Away', 'Back_Over', 'PL_Over', 'PL_Under', 'Back_HA_H', 'PL_HA_H',
       'PL_HA_A', ])

jogos

,Date,League,Time,Home,Away,Odds_H,Odds_A,Over_Line,Odds_Over,Odds_Under,HA_Line,HA_Odds_H,HA_Odds_A,Fixture ID,P(H),P(A),P(O),P(U),P_Diff,Avg_CG_H,DP_CG_H,CV_CG_H,Avg_Pontos_H,DP_Pontos_H,CV_Pontos_H,CG_H_Last_Game,Media_Odds_Geral_H,DP_Odds_Geral_H,CV_Odds_Geral_H,Saldo_Gols_H,Avg_CG_A,DP_CG_A,CV_CG_A,Avg_Pontos_A,DP_Pontos_A,CV_Pontos_A,CG_A_Last_Game,Media_Odds_Geral_A,DP_Odds_Geral_A,CV_Odds_Geral_A,Saldo_Gols_A,Goals_Last_H,Goals_Last_A,Last_Odd_H,Last_Odd_A,MediaCustoGolHome,MediaCustoGolAway,CV_ML,CV_Over,CV_HA,Retornos_BH_Acu,Avg_Retornos_BH,Custo_Retorno_BH,Avg_Porc_BH_Bookie,Avg_Porc_BH_Real,Dist_Porc_BH,Retornos_BA_Acu,Avg_Retornos_BA,Custo_Retorno_BA,Avg_Porc_BA_Bookie,Avg_Porc_BA_Real,Dist_Porc_BA
7843,2024-07-14,Eua Nba,22:30,Toronto Raptors,Denver Nuggets,1.66,2.20,177.5,1.89,1.89,-3.5,1.94,2.49,CKoMEnzQ,0.602410,0.454545,0.529101,0.529101,0.056955,621.332,472.792443,0.760934,0.6,1.341641,2.236068,160.74,5.896,4.600003,0.780191,-33.0,195.274,80.008371,0.409724,1.2,1.643168,1.369306,152.10,2.068,0.412456,0.199447,-46.0,94,78,1.71,1.95,445.454,206.850,0.197843,0.000000,0.175580,-1.36,-0.272,-2.426471,0.296395,0.2,-0.096395,-0.17,-0.034,-35.294118,0.599031,0.5,-0.099031
7844,2024-07-14,Eua Nba,21:00,Atlanta Hawks,San Antonio Spurs,1.72,2.09,171.5,1.80,1.92,-2.5,1.91,2.32,O2uFDQWr,0.581395,0.478469,0.555556,0.520833,0.059864,377.812,305.674338,0.809065,0.0,0.000000,NaN,175.12,3.440,2.602182,0.756448,-67.0,361.854,294.257886,0.813195,2.4,1.341641,0.559017,186.75,3.682,3.369996,0.915262,12.0,88,83,1.99,2.25,331.032,568.964,0.137338,0.045620,0.137075,2.65,0.530,1.358491,0.542055,0.6,0.057945,4.16,0.832,1.310096,0.268533,0.5,0.231467
7845,2024-07-14,Eua Nba,20:30,New Orleans Pelicans,Orlando Magic,3.06,1.37,180.5,1.89,1.89,0.0,0.00,0.00,fmkqJn9L,0.326797,0.729927,0.529101,0.529101,0.056724,202.948,23.605624,0.116314,0.6,1.341641,2.236068,198.32,2.242,0.370095,0.165073,-33.0,208.092,44.065922,0.211762,1.8,1.643168,0.912871,179.14,2.022,0.485098,0.239910,44.0,74,106,2.68,1.69,198.924,250.644,0.539508,0.000000,NaN,-2.98,-0.596,-3.456376,0.539722,0.2,-0.339722,-5.00,-1.000,-0.370000,0.514859,0.2,-0.314859
7846,2024-07-14,Eua Nba,19:00,Washington Wizards,Houston Rockets,2.62,1.48,183.5,1.80,1.92,0.0,0.00,0.00,bLezL8v9,0.381679,0.675676,0.555556,0.520833,0.057355,482.020,426.783269,0.885406,0.6,1.341641,2.236068,168.26,4.032,3.472394,0.861209,-23.0,191.294,42.786612,0.223669,2.4,1.341641,0.559017,142.56,1.670,0.303891,0.181971,48.0,94,99,1.79,1.44,493.458,233.486,0.393220,0.045620,NaN,5.29,1.058,1.531191,0.414436,0.4,-0.014436,-0.44,-0.088,-5.454545,0.549018,0.7,0.150982
7847,2024-07-14,Vietnã Vba,09:30,Saigon Heat,Da Nang Dragons,1.11,5.77,169.5,1.84,1.82,0.0,0.00,0.00,WvoyQlbC,0.900901,0.173310,0.543478,0.549451,0.074211,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000,0.000000,0.000000,0.0,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000,0.000000,0.000000,0.0,87,86,1.36,11.10,0.000,0.000,0.957883,0.007728,NaN,0.00,0.000,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
7848,2024-07-14,Eua Nba,17:00,Los Angeles Clippers,Brooklyn Nets,1.84,1.94,173.5,1.89,1.89,-2.5,2.02,2.14,AaFuaRHk,0.543478,0.515464,0.529101,0.529101,0.058942,261.978,87.592771,0.334352,1.2,1.643168,1.369306,161.04,2.650,0.705727,0.266312,-39.0,344.896,227.380575,0.659273,1.2,1.643168,1.369306,195.94,3.754,2.617094,0.697148,-49.0,88,97,1.83,2.02,188.702,413.834,0.037413,0.000000,0.040795,-3.17,-0.634,-1.324921,0.615007,0.4,-0.215007,-1.32,-0.264,-3.560606,0.369105,0.3,-0.069105
7849,2024-07-14,Eua Nba,16:30,Milwaukee Bucks,Cleveland Cavaliers,2.53,1.51,179.5,1.89,1.89,0.0,0.00,0.00,EFEA4mfS,0.395257,0.662252,0.529101,0.529101,0.057509,333.852,123.539128,0.370042,0.6,1.341641,2.236068,173.55,3.104,1.037078,0.334110,-22.0,547.190,352.084435,0.643441,0.6,1.341641,2.236068,169.85,5.378,3.198026,0.594650,-38.0,89,79,1.95,2.15,219.730,558.684,0.357054,0.000000,NaN,1.58,0.316,4.841772,0.661436,0.5,-0.161436,2.29,0.458,1.113537,0.33794

## Apostas do dia

### Back Home V3 (QCut)

In [9]:
# Duplicando o dataset
df_bh_v3 = jogos.copy()

df_bh_v3 = df_bh_v3[(df_bh_v3['P(H)'] > 0.35) & (df_bh_v3['P(H)'] < 0.65)]
df_bh_v3 = df_bh_v3[(df_bh_v3['CG_H_Last_Game'] > 0) & (df_bh_v3['CG_H_Last_Game'] <= 170)]
df_bh_v3 = df_bh_v3[(df_bh_v3['CG_A_Last_Game'] > 160)]

df_bh_v3.replace([np.inf, -np.inf], 0, inplace=True)

if not df_bh_v3.empty:
    df_bh_v3.to_csv(f'apostas_do_dia/{data_var}/back_home_v3_{data_var}.csv', index=False)

    print('------------ MÉTODO BACK HOME V3 (QCUT) ------------')
    print()

    df_bh_v3['Bet'] = 'Back Home'
    ligs = df_bh_v3['League'].unique().tolist()

    display(df_bh_v3[['Time', 'League', 'Home', 'Away', 'Odds_H', 'Bet']])
    print()
else: print('Sem jogos')

------------ MÉTODO BACK HOME V3 (QCUT) ------------



,Time,League,Home,Away,Odds_H,Bet
7848,17:00,Eua Nba,Los Angeles Clippers,Brooklyn Nets,1.84,Back Home
7875,02:30,Austrália Nbl1 Leste,Sutherland,Bankstown Bruins,1.77,Back Home
7881,17:00,Eua Wnba,Minnesota Lynx F,Indiana Fever F,1.62,Back Home
